<a href="https://colab.research.google.com/github/sw6820/kostat/blob/main/model/h2o_autoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Data

## 0) Logging

In [ ]:
import logging

In [ ]:
logging.basicConfig(filename='automl_1.log',level=logging.DEBUG)

## 1) Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import pandas_profiling

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt 

import re

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kostat/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kostat/2. 모델개발용자료.txt', sep='|', encoding='cp949')
train_co = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kostat/train_nn.csv')
test_co = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kostat/test_common_text.csv')
df_kor_co = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kostat/class_table_chunk_kor.csv')

In [ ]:
train_co.head()

,Unnamed: 0,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,"['카센터', '자동차부분정비', '타이어오일교환']"
1,1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,"['상점내', '일반인', '대상', '채소', '과일판매']"
2,2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,"['절단하여사업체에도매', '공업용고무를가', '합성고무도매']"
3,3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,"['영업점', '일반소비자', '열쇠잠금장치']"
4,4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,"['어린이집', '보호자', '위탁', '취학전아동보육']"


In [ ]:
df_kor_co.head()

,code,chunk_text
0,11,"채소작물 재배업 채소, 화훼작물 및 종묘 재배업 과실작물 재배업 시설작물 재배업 화..."
1,12,양계업 기타 축산업 소 사육업 축산업 양돈업 육우 사육업 가금류 및 조류 사육업 농...
2,13,작물재배 및 축산 복합농업 농업
3,14,"작물재배 관련 서비스업 작물재배 및 축산 관련 서비스업 농업 농산물 건조, 선별 및..."
4,15,수렵 및 관련 서비스업 농업


## 2). Data Preprocessing

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 60.3 MB/s 


In [ ]:
import konlpy
from konlpy.tag import *
hannanum = Hannanum()

In [ ]:
train_co = train_co.fillna('')

In [ ]:
train_co['text'] = train_co['text_obj'].str.cat(train_co[['text_mthd','text_deal']], sep = ' ')

In [ ]:
train_co['text'].value_counts(ascending=False)

개인택시로 일반인을 대상으로 승객운송서비스      37845
용달화물자동차로 고객의 요청으로 화물운송서비스    26016
개별화물자동차로 고객의 요청으로 화물운송서비스    17101
일반화물자동차로 고객의 요청으로 화물운송서비스     2109
교회 기독교계통 종교활동 종교서비스           1040
                             ...  
사업장에서 고객의뢰에의해 실내인테리어             1
가게에서 고객요청으로 컴퓨터수리                1
매장에서 접객시설을갖추고 빵제공                1
사업장에서 고객의의뢰로 화학제품도매              1
사업장에서 접객시설을 갖추고 한식(미역구)판매        1
Name: text, Length: 699218, dtype: int64

In [ ]:
train_co.isnull().sum()

AI_id        0
digit_1      0
digit_2      0
digit_3      0
text_obj     0
text_mthd    0
text_deal    0
text         0
dtype: int64

In [ ]:
train_co['text'].count()

1000000

In [ ]:
df_kor_co['chunk_text'].count()

232

In [ ]:
# train_co['text'] = train_co['text'].apply(hannanum.nouns)

In [ ]:
train_co['text']

0               ['카센터', '자동차부분정비', '타이어오일교환']
1          ['상점내', '일반인', '대상', '채소', '과일판매']
2         ['절단하여사업체에도매', '공업용고무를가', '합성고무도매']
3                  ['영업점', '일반소비자', '열쇠잠금장치']
4            ['어린이집', '보호자', '위탁', '취학전아동보육']
                         ...                 
999995                 ['제품입', '워싱', '청바지워싱']
999996           ['현장', '고객의요청에의해', '실내인테리어']
999997         ['영업점', '일반소비자', '여성의류', '판매']
999998             ['사업장', '일반고객을대상', '필라테스']
999999      ['사업장', '접객시설', '한', '미역구', '판매']
Name: text, Length: 1000000, dtype: object

# 2. H20 AutoML

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(train_co['digit_3'])

In [4]:
model_df = train_co.copy()
model_df['digit_3'] = y
model_df.head()

,Unnamed: 0,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,0,id_0000001,S,95,221,카센터에서,자동차부분정비,타이어오일교환,"['카센터', '자동차부분정비', '타이어오일교환']"
1,1,id_0000002,G,47,126,상점내에서,일반인을 대상으로,채소.과일판매,"['상점내', '일반인', '대상', '채소', '과일판매']"
2,2,id_0000003,G,46,123,절단하여사업체에도매,공업용고무를가지고,합성고무도매,"['절단하여사업체에도매', '공업용고무를가', '합성고무도매']"
3,3,id_0000004,G,47,129,영업점에서,일반소비자에게,열쇠잠금장치,"['영업점', '일반소비자', '열쇠잠금장치']"
4,4,id_0000005,Q,87,212,어린이집,보호자의 위탁을 받아,취학전아동보육,"['어린이집', '보호자', '위탁', '취학전아동보육']"


## 1) XGBoost

In [ ]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

In [6]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 176.2 MB 60 kB/s 
  Created wheel for h2o: filename=h2o-3.36.0.3-py2.py3-none-any.whl size=176244303 sha256=209fe9832bd0eff3f1248f3eb98ecb2a7ddc5bf61aaf785e8ee8473e888f46b6
  Stored in directory: /root/.cache/pip/wheels/b5/71/a5/02087a05e5644158183e1c58eeae3f9356a4d1e80659fb2dfb
Successfully built h2o


In [5]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OXGBoostEstimator

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,35 mins 56 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_unknownUser_yo0xse
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.202 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [15]:
def listToString(lis):
    return ",".join(lis)

model_df['text'] = model_df['text'].apply(listToString)

In [16]:
model_df['text'].head()

0    [,,,',,,카,,,센,,,터,,,',,,,,,, ,,,',,,자,,,동,,,차,...
1    [,,,',,,상,,,점,,,내,,,',,,,,,, ,,,',,,일,,,반,,,인,...
2    [,,,',,,절,,,단,,,하,,,여,,,사,,,업,,,체,,,에,,,도,,,매,...
3    [,,,',,,영,,,업,,,점,,,',,,,,,, ,,,',,,일,,,반,,,소,...
4    [,,,',,,어,,,린,,,이,,,집,,,',,,,,,, ,,,',,,보,,,호,...
Name: text, dtype: object

In [10]:
hf = h2o.H2OFrame(model_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
hf['digit_3'] = hf['digit_3'].asfactor()
predictors = hf.drop(['AI_id','digit_1','digit_2','digit_3']).columns
response = 'digit_3'

In [16]:
predictors

['Unnamed: 0', 'text_obj', 'text_mthd', 'text_deal', 'text']

In [12]:
train, valid = hf.split_frame(ratios=[.8], seed=1234)

In [18]:
industry_xgb = H2OXGBoostEstimator(booster='dart',
                                  normalize_type="tree",
                                  seed=1234,
                                  nfolds=5,
                                  )

In [ ]:
# industry_xgb.train(x=predictors,
#                   y=response,
#                   training_frame=train,
#                   validation_frame=valid)

xgboost Model Build progress: |██████████████████████████████████████████████████

In [ ]:
industry_xgb.metric(metric = 'accuracy', valid = True)

In [14]:
from h2o.automl import H2OAutoML

In [16]:
aml = H2OAutoML(
    max_models=20,
    max_runtime_secs=300,
    seed=1234
)

In [17]:
aml.train(x=predictors,
        y=response,
        training_frame=train,
        validation_frame=valid
)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220323_80147


Model Summary: 


,,number_of_trees
0,,1.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.6672265881070479
RMSE: 0.8168393894193937
LogLoss: 3.9028264672598616
Mean Per-Class Error: 0.9457142353737121
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,...,217,218,219,220,221,222,223,224,Error,Rate
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,1.0,243 / 243
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,159 / 159
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,96 / 96
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,79 / 79
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9 / 9
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,68 / 68
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0 / 0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0 / 0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5 / 5
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,130 / 130



See the whole table with table.as_data_frame()

Top-10 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2022-03-23 08:01:51,0.410 sec,0.0,0.995556,5.416100,0.966930,NaN,NaN,0.995556,5.416100,0.966682,NaN,NaN
1,,2022-03-23 08:14:32,12 min 40.719 sec,1.0,0.816839,3.902826,0.648633,NaN,NaN,0.816910,3.918789,0.648783,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,text_obj.음식점에서,2.036611e+06,1.000000,0.192401
1,"text.[,',개,인,택,시,',,, ,',일,반,인,',,, ,',대,상,',,, ,',승,객,운,송,서,비,스,',]",1.571557e+06,0.771653,0.148467
2,text_obj.용달화물자동차로,1.022360e+06,0.501991,0.096584
3,text_obj.개별화물자동차로,7.032045e+05,0.345282,0.066433
4,text_obj.미용실에서,4.609322e+05,0.226323,0.043545
5,text_deal.종교서비스,4.373352e+05,0.214737,0.041316
6,text_mthd.접객시설을 갖추고,4.342292e+05,0.213212,0.041022
7,text_obj.주점에서,4.076938e+05,0.200182,0.038515
8,text_obj.편의점에서,2.990103e+05,0.146818,0.028248
9,text_obj.학원에서,2.782806e+05,0.136639,0.026289



See the whole table with table.as_data_frame()


In [18]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_1_20220323_80147,0.932534,3.91879,0.81691,0.667341
